In [1]:
""" **Description**
        ``diamondback`` Digital Signal Processing (DSP).

        ``diamondback`` complements Artificial Intelligence (AI) frameworks, by defining
        components which filter, model, and transform data into forms which are
        useful in feature extraction and pattern recognition.

        ``diamondback`` supports applications including cancellation, identification,
        optimization, probabilistic modeling, rate adaptation, and serialization.

    **License**
        `BSD-3C.  <https://github.com/larryturner/diamondback/blob/master/license>`_
        © 2018 - 2024 Larry Turner, Schneider Electric Industries SAS. All rights reserved.

    **Author**
        Larry Turner, Schneider Electric, AI Hub, 2018-01-23.
"""

from diamondback import Log
from diamondback import FirFilter, IirFilter, ComplexBandpassFilter, ComplexExponentialFilter
from diamondback import ComplexFrequencyFilter, DerivativeFilter, GoertzelFilter, IntegralFilter
from diamondback import PolynomialRateFilter, PolyphaseRateFilter, WindowFilter
from diamondback import DiversityModel, GaussianModel, GaussianMixtureModel
from diamondback import ComplexTransform, FourierTransform, PsdTransform, WaveletTransform
from IPython.display import display
from PIL import Image
from typing import Any
import ipywidgets
import math
import matplotlib
import matplotlib.pyplot
import numpy

def label( axis : Any, title : str = '', x : str = '', y : str = '', grid : bool = True ) -> None :

    """ Label plot title and axis, autoscale, and show grid.

        Arguments :
            axis : Any.
            title : str.
            x : str - x label.
            y : str - y label.
            grid : bool.
    """

    axis.set_xlabel( x )
    axis.set_ylabel( y )
    axis.set_title( title )
    axis.grid( grid )
    axis.autoscale( enable = True, axis = 'both', tight = True )


In [2]:
@ipywidgets.interact( )
def interface( style = ipywidgets.Dropdown( description = 'Style', value = 'Hann', options = [ 'Blackman', 'Hamming', 'Hann', 'Kaiser' ], layout = { 'width' : '340px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.1, min = 0.001, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 64, min = 1, max = 4096, step = 1, layout = { 'width' : '420px' } ),
               count = ipywidgets.IntSlider( description = 'Count', value = 1, min = 1, max = 16, step = 1, layout = { 'width' : '420px' } ),
               complement = ipywidgets.IntSlider( description = 'Complement', value = False, min = False, max = True, layout = { 'width' : '420px' } ) ) :

    """ Finite Impulse Response Filter.

        Arguments :
            style : str - in ( 'Blackman', 'Hamming', 'Hann', 'Kaiser' ).
            frequency : float - frequency normalized to Nyquist in ( 0.0, 1.0 ).
            order : int.
            count : int.
            complement : bool.
    """

    try :
        fir_filter = FirFilter( style = style, frequency = frequency, order = order, count = count, complement = complement )
    except Exception as ex :
        Log.write( 'Error', ex )
        return
    widget = [ ipywidgets.Dropdown( description = 'b', value = str( fir_filter.b[ 0 ] ), options = [ str( x ) for x in fir_filter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    _, axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )

    # Coefficients

    length = 1024
    n = numpy.linspace( 0.0, order, order + 1 )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].plot( n, fir_filter.b, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$', r'$\vec{b_{n}}$' )

    # Roots

    plot = [ ]
    p, z = fir_filter.roots( count )
    ii = numpy.where( z.real > 10.0 )[ 0 ]
    z[ ii ] = 10.0 * numpy.sign( z[ ii ].real )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0, 1 ].plot( p.real, p.imag, linestyle = 'None', marker = '+', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( z.real, z.imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$\vec{p_{n}}$', r'$\vec{z_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Roots$', r'$real$', r'$imag$' )

    # Response

    h, f = fir_filter.response( count = count )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    ii, jj, = int( numpy.round( ( len( h ) // 2 ) * ( 1.0 - frequency ) ) ), int( numpy.round( ( len( h ) // 2 ) * ( 1.0 + frequency ) ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( -frequency, h[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 0 ].plot( frequency, h[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )

    # Delay

    d, f = fir_filter.delay( count = count )
    d = numpy.round( d * 100.0 ) / 100.0
    axis[ 1, 1 ].plot( -frequency, d[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 1 ].plot( frequency, d[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 1 ].plot( f, d, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 1, 1 ], r'$Delay$', r'$f$', r'$\tau_{f}\ n$' )
    matplotlib.pyplot.show( )


interactive(children=(Dropdown(description='Style', index=2, layout=Layout(width='340px'), options=('Blackman'…

In [3]:
@ipywidgets.interact( )
def interface( style = ipywidgets.Dropdown( description = 'Style', value = 'Butterworth', options = [ 'Bessel', 'Butterworth', 'Chebyshev' ], layout = { 'width' : '340px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.1, min = 0.001, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 6, min = 1, max = 16, step = 1, layout = { 'width' : '420px' } ),
               count = ipywidgets.IntSlider( description = 'Count', value = 1, min = 1, max = 16, step = 1, layout = { 'width' : '420px' } ),
               complement = ipywidgets.IntSlider( description = 'Complement', value = False, min = False, max = True, layout = { 'width' : '420px' } ) ) :

    """ Infinite Impulse Response Filter.

        Arguments :
            style : str - in ( 'Bessel', 'Butterworth', 'Chebyshev' ).
            frequency : float - frequency normalized to Nyquist in ( 0.0, 1.0 ).
            order : int.
            count : int.
            complement : bool.
    """

    iir_filter = IirFilter( style = style, frequency = frequency, order = order, count = count, complement = complement )
    widget = [ ipywidgets.Dropdown( description = 'a', value = str( iir_filter.a[ 0 ] ), options = [ str( x ) for x in iir_filter.a ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = 'b', value = str( iir_filter.b[ 0 ] ), options = [ str( x ) for x in iir_filter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    _, axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )

    # Coefficients

    plot = [ ]
    length = 1024
    n = numpy.linspace( 0.0, order, order + 1 )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].bar( n, iir_filter.a, color = [ 0.8, 0.2, 0.2 ], width = 0.006 * ( order + 1 ) )
    axis[ 0, 0 ].bar( n, iir_filter.b, color = [ 0.2, 0.2, 0.8 ], width = 0.006 * ( order + 1 ) )
    plot.append( axis[ 0, 0 ].plot( n, iir_filter.a, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 0 ].plot( n, iir_filter.b, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 0 ].legend( plot, ( r'$\vec{a_{n}}$', r'$\vec{b_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$' )

    # Roots

    plot = [ ]
    p, z = iir_filter.roots( count )
    ii = numpy.where( z.real > 10.0 )[ 0 ]
    z[ ii ] = 10.0 * numpy.sign( z[ ii ].real )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0, 1 ].plot( p.real, p.imag, linestyle = 'None', marker = '+', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( z.real, z.imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$\vec{p_{n}}$', r'$\vec{z_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Roots$', r'$real$', r'$imag$' )

    # Response

    h, f = iir_filter.response( count = count )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    ii, jj, = int( numpy.round( ( len( h ) // 2 ) * ( 1.0 - frequency ) ) ), int( numpy.round( ( len( h ) // 2 ) * ( 1.0 + frequency ) ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( -frequency, h[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 0 ].plot( frequency, h[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )

    # Delay

    d, f = iir_filter.delay( count = count )
    d = numpy.round( d * 100.0 ) / 100.0
    axis[ 1, 1 ].plot( -frequency, d[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 1 ].plot( frequency, d[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 1 ].plot( f, d, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 1, 1 ], r'$Delay$', r'$f$', r'$\tau_{f}\ n$' )
    matplotlib.pyplot.show( )


interactive(children=(Dropdown(description='Style', index=1, layout=Layout(width='340px'), options=('Bessel', …

In [4]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.05, 0.1 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               derivative = ipywidgets.IntSlider( description = 'Derivative', value = 1, min = 1, max = 3, step = 1, layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 2, min = 0, max = 8, step = 2, layout = { 'width' : '420px' } ) ) :

    """ Derivative Filter.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            derivative : int.
            order : int.
    """

    order = int( max( min( order, 2.0 ** ( derivative + 1 ) ), 2.0 ** ( derivative - 1 ) ) )
    derivative_filter = DerivativeFilter( derivative, order )
    widget = [ ipywidgets.Dropdown( description = 'b', value = str( derivative_filter.b[ 0 ] ), options = [ str( x ) for x in derivative_filter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    _, axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )

    # Coefficients

    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    n = numpy.linspace( 0.0, order, order + 1 )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].bar( n, derivative_filter.b, color = [ 0.2, 0.2, 0.8 ], width = 0.006 * ( order + 1 ) )
    axis[ 0, 0 ].plot( n, derivative_filter.b, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$', r'$\vec{b_{n}}$' )

    # Roots

    plot = [ ]
    p, z = derivative_filter.roots( )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0, 1 ].plot( p.real, p.imag, linestyle = 'None', marker = '+', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( z.real, z.imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$\vec{p_{n}}$', r'$\vec{z_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Roots$', r'$real$', r'$imag$' )

    # Response

    h, f = derivative_filter.response( )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )

    # Filter

    plot = [ ]
    length = 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) ).real
    q = numpy.random.rand( len( x ) ) - 0.5
    x += q * noise / q.std( )
    y = derivative_filter.filter( x )
    n = numpy.linspace( 0, length - 1, length )
    plot.append( axis[ 1, 1 ].plot( n, x, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ 1, 1 ].plot( n, y, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    axis[ 1, 1 ].legend( plot, ( r'$x_{n}$', r'$y_{n}$' ), loc = 'upper right' )
    label( axis[ 1, 1 ], r'$Filter$', r'$n$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(-0.05, 0.1), description='Incident', layout=Layout(width='420px'…

In [5]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.05, 0.1 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1, min = 0, max = 4, step = 1, layout = { 'width' : '420px' } ) ) :

    """ Integral Filter.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            order : int.
    """

    integral_filter = IntegralFilter( order )
    widget = [ ipywidgets.Dropdown( description = 'a', value = str( integral_filter.a[ 0 ] ), options = [ str( x ) for x in integral_filter.a ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = 'b', value = str( integral_filter.b[ 0 ] ), options = [ str( x ) for x in integral_filter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    _, axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )

    # Coefficients

    plot = [ ]
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    n = numpy.linspace( 0.0, max( order, 1 ), max( order, 1 ) + 1 )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].bar( n, integral_filter.a, color = [ 0.8, 0.2, 0.2 ], width = 0.006 * ( order + 1 ) )
    axis[ 0, 0 ].bar( n, integral_filter.b, color = [ 0.2, 0.2, 0.8 ], width = 0.006 * ( order + 1 ) )
    plot.append( axis[ 0, 0 ].plot( n, integral_filter.a, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 0 ].plot( n, integral_filter.b, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 0 ].legend( plot, ( r'$\vec{a_{n}}$', r'$\vec{b_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$' )

    # Roots

    plot = [ ]
    p, z = integral_filter.roots( )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0, 1 ].plot( p.real, p.imag, linestyle = 'None', marker = '+', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( z.real, z.imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$\vec{p_{n}}$', r'$\vec{z_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Roots$', r'$real$', r'$imag$' )

    # Response

    h, f = integral_filter.response( )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )

    # Filter

    plot = [ ]
    length = 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) ).real
    q = numpy.random.rand( len( x ) ) - 0.5
    x += q * noise / q.std( )
    y = integral_filter.filter( x )
    n = numpy.linspace( 0, length - 1, length )
    plot.append( axis[ 1, 1 ].plot( n, x, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ 1, 1 ].plot( n, y, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    axis[ 1, 1 ].legend( plot, ( r'$x_{n}$', r'$y_{n}$' ), loc = 'upper right' )
    label( axis[ 1, 1 ], r'$Filter$', r'$n$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(-0.05, 0.1), description='Incident', layout=Layout(width='420px'…

In [6]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ 0.0, 0.1 ], min = -1.0, max = 0.99, step = 0.01, readout_format = '0.2f', layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               rate = ipywidgets.FloatSlider( description = 'Rate', value = 1.0, min = 0.01, max = 256.0, step = 0.01, layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1, min = 1, max = 3, step = 1, layout = { 'width' : '420px' } ) ) :

    """ Polynomial Rate Filter.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - noise deviation of incident signal.
            rate : float - ratio of effective frequency in [ 1.0, inf ).
            order : int.
    """

    _, axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 36, 9 ) )

    # Filter

    plot = [ ]
    polynomial_rate_filter = PolynomialRateFilter( rate, order )
    phase, length = 0.5, 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) ).real
    q = numpy.random.rand( len( x ) ) - 0.5
    x += q * noise / q.std( )
    y = polynomial_rate_filter.filter( x )
    plot.append( axis[ 0 ].plot( numpy.linspace( 0, len( x ) - 1, len( x ) ), x, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ 0 ].plot( numpy.linspace( 0, len( y ) - 1, len( y ) ), y, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    axis[ 0 ].legend( plot, ( r'$x_{n}$', r'$y_{n}$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Filter$', r'$n$' )
    axis[ 1 ].set_visible( False )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(0.0, 0.1), description='Incident', layout=Layout(width='420px'),…

In [7]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ 0.0, 0.1 ], min = -1.0, max = 0.99, step = 0.01, readout_format = '0.2f', layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               rate = ipywidgets.FloatSlider( description = 'Rate', value = 1.0, min = 0.01, max = 256.0, step = 0.01, layout = { 'width' : '420px' } ) ) :

    """ Polyphase Rate Filter.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            rate : float - ratio of effective frequency in ( 0.0, 256.0 ].
    """

    polyphase_rate_filter = PolyphaseRateFilter( rate )
    widget = [ ipywidgets.Dropdown( description = 'Index', value = str( polyphase_rate_filter.b.shape[ 0 ] / rate ), options = [ str( polyphase_rate_filter.b.shape[ 0 ] / rate ) ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = 'Order', value = str( polyphase_rate_filter.b.shape[ 0 ] ), options = [ str( polyphase_rate_filter.b.shape[ 0 ] ), str( polyphase_rate_filter.b.shape[ 1 ] ) ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    _, axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )

    # Coefficients

    frequency, phase, length = 0.85, 0.5, 1024
    n = numpy.linspace( 0.0, polyphase_rate_filter.b.shape[ 1 ] - 1, polyphase_rate_filter.b.shape[ 1 ] )
    axis[ 0, 0 ].plot( n, numpy.zeros( polyphase_rate_filter.b.shape[ 1 ] ), color = [ 0.0, 0.0, 0.0 ], linewidth = 0.5 )
    for ii in range( 0, polyphase_rate_filter.b.shape[ 0 ], 4 ) :
        axis[ 0, 0 ].plot( n, polyphase_rate_filter.b[ ii, : ], color = [ 0.2, 0.2, 0.8 ], linewidth = 0.25 )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$', r'$\vec{b_{n}}$' )

    # Filter

    plot = [ ]
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) ).real
    q = numpy.random.rand( len( x ) ) - 0.5
    x += q * noise / q.std( )
    y = polyphase_rate_filter.filter( x )
    plot.append( axis[ 0, 1 ].plot( numpy.linspace( 0, len( x ) - 1, len( x ) ), x, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( numpy.linspace( 0, len( y ) - 1, len( y ) ), y, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$x_{n}$', r'$y_{n}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Filter$', r'$n$' )
    for ii in range( 0, polyphase_rate_filter.b.shape[ 0 ], 2 ) :
        fir_filter = FirFilter( b = polyphase_rate_filter.b[ ii, : ] )
        h, f = fir_filter.response( )
        h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
        ii, jj, = int( numpy.round( len( h ) // 2 * ( 1.0 - frequency ) ) ), int( numpy.round( len( h ) // 2 * ( 1.0 + frequency ) ) )
        axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 0.25 )
        d, f = fir_filter.delay( )
        d = numpy.maximum( numpy.minimum( d, polyphase_rate_filter.b.shape[ 1 ] // 2 + 0.6 ), polyphase_rate_filter.b.shape[ 1 ] // 2 - 0.6 )
        axis[ 1, 1 ].plot( f, d, 'b', linewidth = 0.25 )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( -frequency, h[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 0 ].plot( frequency, h[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )
    label( axis[ 1, 1 ], r'$Delay$', r'$f$', r'$\tau_{f}\ n$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(0.0, 0.1), description='Incident', layout=Layout(width='420px'),…

In [8]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.05, 0.05 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               neutral = ipywidgets.IntSlider( description = 'Neutral', value = True, min = False, max = True, layout = { 'width' : '420px' } ),
               inverse = ipywidgets.IntSlider( description = 'Inverse', value = False, min = False, max = True, layout = { 'width' : '420px' } ) ) :

    """ Complex Transform.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            neutral : bool.
            inverse : bool.
    """

    _, axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )

    # Incident, Transform

    plot = [ ]
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    n = numpy.linspace( 0.0, length - 1, length )
    v = ComplexTransform.transform( x, neutral )
    epoch, terminus, ii, jj = r'$Incident$', r'$Transform$', 0, 1
    if ( inverse ) :
        x = ComplexTransform.transform( v, neutral )
        epoch, terminus, ii, jj = r'$Transform$', r'$Incident$', 1, 0
    plot.append( axis[ ii ].plot( n, v[ 0, : ], color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ ii ].plot( n, v[ 1, : ], color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ ii ].plot( n, v[ 2, : ], color = [ 0.8, 0.2, 0.2 ], linewidth = 1.0 )[ 0 ] )
    axis[ ii ].legend( plot, ( r'$x_{0,n}$', r'$x_{1,n}$', r'$x_{2,n}$' ), loc = 'upper right' )
    label( axis[ ii ], epoch, r'$n$' )
    plot = [ ]
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ jj ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ jj ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ jj ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ jj ].plot( x.real, x.imag, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    plot.append( axis[ jj ].plot( x[ 0 ].real, x[ 0 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ jj ].plot( x[ -1 ].real, x[ -1 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ jj ].legend( plot, ( r'$x_{0}$', r'$x_{N-1}$' ), loc = 'upper right' )
    label( axis[ jj ], terminus, r'$real$', r'$imag$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(-0.05, 0.05), description='Incident', layout=Layout(width='420px…

In [17]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ 0.1, 0.1 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.1, min = -1.0, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ),
               rate = ipywidgets.FloatSlider( description = 'Rate', value = 0.01, min = 0.0001, max = 1.0, step = 0.0001, readout_format = '0.4f', layout = { 'width' : '420px' } ) ) :

    """ Complex Bandpass Filter.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            frequency : float - frequency normalized to Nyquist in [ -1.0, 1.0 ).
            rate : float.
    """

    _, axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )

    # Coefficients

    complex_bandpass_filter = ComplexBandpassFilter( frequency, rate )
    phase, count, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 128, 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    y, e, b = complex_bandpass_filter.filter( x )
    n = numpy.linspace( 0.0, length - 1, length )
    axis[ 0, 0 ].plot( n, numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].plot( n, b.real, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )
    axis[ 0, 0 ].plot( n, b.imag, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )
    axis[ 0, 0 ].plot( n, abs( b ), color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$', r'$b_{n}$' )

    # Coefficients

    plot = [ ]
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( b.real, b.imag, linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.2, 0.2, 0.8 ], markersize = 4.0 )
    plot.append( axis[ 0, 1 ].plot( b[ 0 ].real, b[ 0 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( b[ -1 ].real, b[ -1 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$b_{0}$', r'$b_{N-1}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Coefficients$', r'$real$', r'$imag$' )

    # Response

    plot = [ ]
    h, f = numpy.zeros( count ), numpy.linspace( max( incident[ 0 ] - 0.05, -1.0 ), min( incident[ 1 ] + 0.05, 1.0 - 2.0 / count ), count )
    for ii in range( 0, len( f ) ) :
        u = ComplexExponentialFilter( phase ).filter( numpy.ones( length ) * f[ ii ] )
        complex_bandpass_filter.reset( 0.0 )
        v = complex_bandpass_filter.filter( u )[ 0 ]
        h[ ii ] = min( numpy.mean( abs( v[ -32 : -1 ] ) ), 1.0 )
    h, k = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) ), 20.0 * numpy.log10( numpy.maximum( abs( 1.0 - h ), numpy.finfo( float ).eps ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1, 0 ].plot( f, k, color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 1, 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
    axis[ 1, 0 ].legend( plot, ( r'$y_{f}$', r'$e_{f}$' ), loc = 'upper right' )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )

    # Filter

    plot = [ ]
    axis[ 1, 1 ].plot( n, x.real, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )
    axis[ 1, 1 ].plot( n, y.real, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )
    axis[ 1, 1 ].plot( n, e.real, color = [ 0.8, 0.2, 0.2 ], linewidth = 1.0 )
    plot.append( axis[ 1, 1 ].plot( n, abs( x ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1, 1 ].plot( n, abs( y ), color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1, 1 ].plot( n, abs( e ), color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 1, 1 ].legend( plot, ( r'$H_{x,f}$', r'$H_{y,f}$', r'$H_{e,f}$' ), loc = 'upper right' )
    label( axis[ 1, 1 ], r'$Filter$', r'$n$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(0.1, 0.1), description='Incident', layout=Layout(width='420px'),…

In [10]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.25, 0.75 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.0, min = -1.0, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ),
               rate = ipywidgets.FloatSlider( description = 'Rate', value = 0.25, min = 0.001, max = 1.0, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ) ) :

    """ Complex Frequency Filter.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            frequency : float - frequency normalized to Nyquist in [ -1.0, 1.0 ).
            rate : float.
    """

    _, axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )

    # Coefficients

    plot = [ ]
    complex_frequency_filter = ComplexFrequencyFilter( frequency, rate )
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    f = numpy.linspace( incident[ 0 ], incident[ 1 ], length )
    x = ComplexExponentialFilter( phase ).filter( f )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    y, e, b = complex_frequency_filter.filter( x )
    n = numpy.linspace( 0.0, length - 1, length )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 0.5 )
    axis[ 0 ].plot( b.real, b.imag, linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.2, 0.2, 0.8 ], markersize = 4.0 )
    plot.append( axis[ 0 ].plot( b[ 0 ].real, b[ 0 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0 ].plot( b[ -1 ].real, b[ -1 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0 ].legend( plot, ( r'$b_0$', r'$b_{N-1}$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Coefficients$', r'$real$', r'$imag$' )

    # Filter

    plot = [ ]
    axis[ 1 ].plot( n, numpy.maximum( numpy.minimum( e.real, 1.0 ), -1.0 ), color = [ 0.8, 0.2, 0.2 ], linewidth = 1.0 )
    plot.append( axis[ 1 ].plot( n, f, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( n, y, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( n, abs( e ), color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 1 ].legend( plot, ( r'$f_{x,n}$', r'$y_n$', r'$e_n$' ), loc = 'upper right' )
    label( axis[ 1 ], r'$Filter$', r'$n$', r'$f_n$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(-0.25, 0.75), description='Incident', layout=Layout(width='420px…

In [11]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ 0.05, 0.1 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.01, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               style = ipywidgets.Dropdown( description = 'Style', value = 'Hann', options = [ 'Blackman', 'Hamming', 'Hann', 'Kaiser' ], layout = { 'width' : '340px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1023, min = 1, max = 4096, step = 1, layout = { 'width' : '420px' } ),
               normal = ipywidgets.IntSlider( description = 'Normal', value = True, min = False, max = True, layout = { 'width' : '420px' } ),
               index = ipywidgets.IntSlider( description = 'Index', value = 64, min = 1, max = 4096, step = 1, layout = { 'width' : '420px' } ),
               spectrogram = ipywidgets.IntSlider( description = 'Spectrogram', value = True, min = False, max = True, layout = { 'width' : '420px' } ) ) :

    """ Window Filter, Fourier Transform, and Power Spectral Density Transform.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            style : str - in ( 'Blackman', 'Hamming', 'Hann', 'Kaiser' ).
            normal : bool.
            index : int.
            spectrogram : bool.
    """

    # Window Filter

    window_filter = WindowFilter( style, order, normal )
    widget = [ ipywidgets.Dropdown( description = 'b', value = str( window_filter.b[ 0 ] ), options = [ str( x ) for x in window_filter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    _, axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )

    # Fourier Transform

    plot = [ ]
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, ( order + 1 ) * 16
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) )
    for ii in ( 0, int( ( len( x ) - len( window_filter.b ) ) / 2 ), len( x ) - len( window_filter.b ) ) :
        q = numpy.random.rand( len( x[ ii : ] ) ) - 0.5 + 1j * ( numpy.random.rand( len( x[ ii : ] ) ) - 0.5 )
        h, f = FourierTransform.transform( x[ ii : ] + q * noise / q.std( ), window_filter.b, False )
        h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
        plot.append( axis[ 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    axis[ 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), 'k' )
    axis[ 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], len( f ) ), numpy.zeros( len( f ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
    axis[ 0 ].legend( plot, ( r'$H^{F}_{x,f}$', ), loc = 'upper right' )
    label( axis[ 0 ], r'$Fourier$', r'$f$', r'$H_{f}\ dB$' )

    # Power Spectral Density Transform

    h, f = PsdTransform.transform( x, window_filter.b, index, spectrogram )
    h = 10.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    if ( not spectrogram ) :
        axis[ 1 ].plot( f, -3.0 * numpy.ones( len( h ) ), 'k' )
        plot.append( axis[ 1 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
        axis[ 1 ].plot( numpy.linspace( max( incident[ 0 ], 0.0 ), max( abs( incident[ 1 ] ), 0.0 ), length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
        axis[ 1 ].legend( plot, ( r'$H^{P}_{x,f}$', ), loc = 'upper right' )
        label( axis[ 1 ], r'$Power$', r'$f$', r'$H_{f}\ dB$' )
    else :
        color = matplotlib.pyplot.get_cmap( 'gist_heat' )
        n = numpy.array( range( 0, h.shape[ 0 ] ) )
        axis[ 1 ].pcolor( f, n, h, cmap = color )
        axis[ 1 ].set_xlim( n[ 0 ], n[ -1 ] )
        label( axis[ 1 ], r'$Power$', r'$f$', r'$n$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(0.05, 0.1), description='Incident', layout=Layout(width='420px')…

In [12]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.1, 0.15 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               style = ipywidgets.Dropdown( description = 'Style', value = 'Hann', options = [ 'Blackman', 'Hamming', 'Hann', 'Kaiser' ], layout = { 'width' : '340px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1023, min = 1, max = 4096, step = 1, layout = { 'width' : '420px' } ),
               normal = ipywidgets.IntSlider( description = 'Normal', value = True, min = False, max = True, layout = { 'width' : '420px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.1, min = -1.0, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ) ) :

    """ Goertzel Filter, Window Filter.

        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] normalized to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            order : int.
            normal : bool.
            frequency : float - frequency normalized to Nyquist in [ -1.0, 1.0 ).
    """

    window_filter = WindowFilter( style, order, normal )
    goertzelfilter = GoertzelFilter( window_filter.b, frequency )
    widget = [ ipywidgets.Dropdown( description = 'a', value = str( goertzelfilter.a[ 0 ] ), options = [ str( x ) for x in goertzelfilter.a ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = 'b', value = str( goertzelfilter.b[ 0 ] ), options = [ str( x ) for x in goertzelfilter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    _, axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )

    # Incident

    plot = [ ]
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, ( order + 1 ) * 32
    f = numpy.linspace( incident[ 0 ], incident[ 1 ], length )
    x = ComplexExponentialFilter( phase ).filter( f )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    h = goertzelfilter.filter( x )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    n = numpy.linspace( 0.0, len( h ) - 1, len( h ) )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )
    axis[ 0 ].plot( x.real, x.imag, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )
    plot.append( axis[ 0 ].plot( x[ 0 ].real, x[ 0 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0 ].plot( x[ -1 ].real, x[ -1 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0 ].legend( plot, ( r'$x_{0}$', r'$x_{N-1}$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Incident$', r'$real$', r'$imag$' )

    # Filter

    axis[ 1 ].plot( n, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 1 ], r'$Filter$', r'$n$', r'$H_{f,n}\ dB$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatRangeSlider(value=(-0.1, 0.15), description='Incident', layout=Layout(width='420px'…

In [18]:
@ipywidgets.interact( )
def interface( noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.25, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               style = ipywidgets.Dropdown( description = 'Style', value = 'Euclidean', options = [ 'Chebyshev', 'Euclidean', 'Geometric', 'Manhattan' ], layout = { 'width' : '340px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 4, min = 2, max = 64, step = 1, layout = { 'width' : '420px' } ) ) :

    """ Diversity Model.

        Arguments :
            noise : float - incident signal deviation.
            style : str - in ( 'Chebyshev', 'Euclidean', 'Geometric', 'Manhattan' ).
            order : int.
    """

    diversity_model = DiversityModel( style, order )
    _, axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )

    # Diversity

    length = 1024
    n = numpy.linspace( 0.0, length - 1, length )
    x = numpy.concatenate( ( [ numpy.polyval( numpy.random.rand( 2 ), n ) ], [ numpy.polyval( numpy.random.rand( 3 ), n ) + numpy.random.rand( 1 )[ 0 ] - 0.5 ] ) )
    x = ( x.T / x.std( 1 ) )
    q = numpy.random.rand( x.shape[ 0 ], x.shape[ 1 ] ) - 0.5
    x += q * noise / q.std( )
    d = diversity_model.learn( x )
    p = numpy.polyfit( diversity_model.s[ :, 0 ], diversity_model.s[ :, 1 ], 2 )
    y = numpy.polyval( p, x[ :, 0 ] )
    axis[ 0 ].plot( n, d, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 0 ], r'$Diversity$', r'$n$', r'$d_{n}$' )

    # Model

    plot = [ ]
    ii = numpy.argsort( x[ :, 0 ] )
    plot.append( axis[ 1 ].plot( x[ ii, 0 ], x[ ii, 1 ], linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.2, 0.2, 0.8 ], markersize = 4.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( x[ ii, 0 ], y[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.8, 0.2, 0.2 ], markersize = 4.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( diversity_model.s[ :, 0 ], diversity_model.s[ :, 1 ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )[ 0 ] )
    axis[ 1 ].legend( plot, ( r'$x_{1,n}$', r'$y_{n}$', r'$s_{N-1}$' ), loc = 'upper right' )
    label( axis[ 1 ], r'$Model$', r'$x_{0,n}$', r'$x_{1,n}$' )
    matplotlib.pyplot.show( )


interactive(children=(FloatSlider(value=0.25, description='Noise', layout=Layout(width='420px'), max=0.5, step…

In [14]:
@ipywidgets.interact( )
def interface( noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.01, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               regularize = ipywidgets.FloatSlider( description = 'Regularize', value = 1.0e-1, min = 0.0, max = 1.0, step = 0.01, layout = { 'width' : '420px' } ) ) :

    """ Gaussian Model.

        Arguments :
            noise : float - incident signal deviation.
            regularize : float - regularize.
    """

    gaussian_model = GaussianModel( regularize )
    _, axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )

    # Incident

    plot = [ ]
    length = 1024
    c = matplotlib.pyplot.get_cmap( 'jet' )
    u = numpy.random.randn( 4, 2 ) * 1.0e-1
    for ii in range( 0, 2 ) :
        for jj in range( 0, 2 ) :
            u[ ii * 2 + jj, : ] += [ ( ii * 2.0 - 1.0 ) * 0.25, ( jj * 2.0 - 1.0 ) * 0.25 ]
    x = numpy.random.randn( length, 2 ) * 5.0e-2
    q = numpy.random.rand( x.shape[ 0 ], x.shape[ 1 ] ) - 0.5
    x += q * noise / q.std( )
    y = numpy.random.randint( 0, u.shape[ 0 ], length )
    for ii in range( 0, u.shape[ 0 ] ) :
        jj = numpy.where( y == ii )[ 0 ]
        x[ jj ] += u[ ii ]
    gaussian_model.learn( x, y )
    i = gaussian_model.predict( x )
    i, v = numpy.array( [ numpy.where( i[ ii ] == y[ ii ] )[ 0 ][ 0 ] for ii in range( 0, len( y ) ) ] ), i[ :, 0 ]
    for ii in range( 0, u.shape[ 0 ] ) :
        jj = numpy.where( y == ii )[ 0 ]
        plot.append( axis[ 0 ].plot( x[ jj, 0 ], x[ jj, 1 ], linestyle = 'None', marker = 'o', markeredgecolor = 'k', markeredgewidth = 2.0, markerfacecolor = c( ii * 0.2 + 0.1 ), markersize = 6.0 )[ 0 ] )
    for ii in range( 0, u.shape[ 0 ] ) :
        jj = numpy.where( y == ii )[ 0 ]
        kk = numpy.where( v[ jj ] != y[ jj ] )[ 0 ]
        if ( len( kk ) ) :
            plot.append( axis[ 0 ].plot( x[ jj ][ kk, 0 ], x[ jj ][ kk, 1 ], linestyle = 'None', marker = 'o', markeredgecolor = None, markeredgewidth = 2.0, markerfacecolor = 'r', markersize = 6.0 )[ 0 ] )
    axis[ 0 ].legend( plot, ( r'$y_{0}$', r'$y_{1}$', r'$y_{2}$', r'$y_{3}$', r'$e$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Incident$', r'$x_{0,n}$', r'$x_{1,n}$' )

    # Position

    ii = [ ii for ii in range( 0, max( y ) + 1 ) ]
    p = numpy.bincount( i ) / len( i )
    n = max( y ) + 1
    if ( len( p ) < n ) :
        p = numpy.concatenate( ( p, [ 0.0 ] * ( n - len( p ) ) ) )
    v = numpy.array( [ p[ : jj + 1 ].sum( ) for jj in range( 0, len( p ) ) ] )
    axis[ 1 ].bar( ii, p, label = r'$p_{i}$', alpha = 0.7, color = c( 0.0 ), align = 'center', width = 0.5 )
    axis[ 1 ].plot( ii, v, label = r'$c_{i}$', alpha = 0.5, color = c( 0.25 ), linewidth = 6.0 )
    for jj in ii :
        axis[ 1 ].plot( jj, v[ jj ], marker = 'o', markeredgecolor = 'w', markerfacecolor = 'r', markersize = 10.0 )
    axis[ 1 ].legend( loc = 'upper right' )
    axis[ 1 ].grid( True )
    axis[ 1 ].autoscale( enable = True, axis = 'both', tight = True )
    axis[ 1 ].set_xlabel( r'$i$' )
    axis[ 1 ].set_xlim( min( ii ) - 0.5, max( ii ) + 0.5 )
    axis[ 1 ].set_ylim( 0.0, 1.0 )
    matplotlib.pyplot.xticks( ticks = sorted( set( y ) ) )
    label( axis[ 1 ], r'$Position$', r'$i$', r'$p_{i}$' )


interactive(children=(FloatSlider(value=0.01, description='Noise', layout=Layout(width='420px'), max=0.5, step…

In [15]:
@ipywidgets.interact( )
def interface( noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.01, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               mixture = ipywidgets.IntSlider( description = 'Mixture', value = 1, min = 1, max = 20, step = 1, layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 2, min = 1, max = 20, step = 1, layout = { 'width' : '420px' } ),
               index = ipywidgets.IntSlider( description = 'Index', value = 50, min = 10, max = 100, step = 1, layout = { 'width' : '420px' } ),
               regularize = ipywidgets.FloatSlider( description = 'Regularize', value = 1.0e-1, min = 0.0, max = 1.0, step = 0.01, layout = { 'width' : '420px' } ) ) :

    """ Gaussian Mixture Model.

        Arguments :
            noise : float - incident signal deviation.
            mixture : int - incident mixture distributions per class.
            order : int - model mixture distributions per class.
            index : int - iterations.
            regularize : float - regularize.
    """

    gaussian_mixture_model = GaussianMixtureModel( order, index, regularize )
    _, axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )

    # Incident

    plot = [ ]
    length = 1024
    c = matplotlib.pyplot.get_cmap( 'jet' )
    u = numpy.random.randn( 4, 2 ) * 5.0e-2
    for ii in range( 0, 2 ) :
        for jj in range( 0, 2 ) :
            u[ ii * 2 + jj, : ] += [ ( ii * 2.0 - 1.0 ) * 0.25, ( jj * 2.0 - 1.0 ) * 0.25 ]
    x = numpy.random.randn( length, 2 ) * 1.0e-1
    q = numpy.random.rand( x.shape[ 0 ], x.shape[ 1 ] ) - 0.5
    x += q * noise / q.std( )
    y = numpy.random.randint( 0, u.shape[ 0 ], length )
    for ii in range( 0, u.shape[ 0 ] ) :
        jj = numpy.where( y == ii )[ 0 ]
        x[ jj ] += u[ ii ]
        z = x[ jj ]
        for kk in range( 1, mixture ) :
            z[ kk : : mixture ] += numpy.random.randn( 2 ) * 2.5e-2
        x[ jj ] = z
    gaussian_mixture_model.learn( x, y )
    i = gaussian_mixture_model.predict( x )
    i, v = numpy.array( [ numpy.where( i[ ii ] == y[ ii ] )[ 0 ][ 0 ] for ii in range( 0, len( y ) ) ] ), i[ :, 0 ]
    for ii in range( 0, u.shape[ 0 ] ) :
        jj = numpy.where( y == ii )[ 0 ]
        plot.append( axis[ 0 ].plot( x[ jj, 0 ], x[ jj, 1 ], linestyle = 'None', marker = 'o', markeredgecolor = 'k', markeredgewidth = 2.0, markerfacecolor = c( ii * 0.2 + 0.1 ), markersize = 6.0 )[ 0 ] )
    for ii in range( 0, u.shape[ 0 ] ) :
        jj = numpy.where( y == ii )[ 0 ]
        kk = numpy.where( v[ jj ] != y[ jj ] )[ 0 ]
        if ( len( kk ) ) :
            plot.append( axis[ 0 ].plot( x[ jj ][ kk, 0 ], x[ jj ][ kk, 1 ], linestyle = 'None', marker = 'o', markeredgecolor = None, markeredgewidth = 2.0, markerfacecolor = 'r', markersize = 6.0 )[ 0 ] )
    axis[ 0 ].legend( plot, ( r'$y_{0}$', r'$y_{1}$', r'$y_{2}$', r'$y_{3}$', r'$e$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Incident$', r'$x_{0,n}$', r'$x_{1,n}$' )

    # Position

    ii = [ ii for ii in range( 0, max( y ) + 1 ) ]
    p = numpy.bincount( i ) / len( i )
    n = max( y ) + 1
    if ( len( p ) < n ) :
        p = numpy.concatenate( ( p, [ 0.0 ] * ( n - len( p ) ) ) )
    v = numpy.array( [ p[ : jj + 1 ].sum( ) for jj in range( 0, len( p ) ) ] )
    axis[ 1 ].bar( ii, p, label = r'$p_{i}$', alpha = 0.7, color = c( 0.0 ), align = 'center', width = 0.5 )
    axis[ 1 ].plot( ii, v, label = r'$c_{i}$', alpha = 0.5, color = c( 0.25 ), linewidth = 6.0 )
    for jj in ii :
        axis[ 1 ].plot( jj, v[ jj ], marker = 'o', markeredgecolor = 'w', markerfacecolor = 'r', markersize = 10.0 )
    axis[ 1 ].legend( loc = 'upper right' )
    axis[ 1 ].grid( True )
    axis[ 1 ].autoscale( enable = True, axis = 'both', tight = True )
    axis[ 1 ].set_xlabel( r'$i$' )
    axis[ 1 ].set_xlim( min( ii ) - 0.5, max( ii ) + 0.5 )
    axis[ 1 ].set_ylim( 0.0, 1.0 )
    matplotlib.pyplot.xticks( ticks = sorted( set( y ) ) )
    label( axis[ 1 ], r'$Position$', r'$i$', r'$p_{i}$' )


interactive(children=(FloatSlider(value=0.01, description='Noise', layout=Layout(width='420px'), max=0.5, step…

In [16]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.Dropdown( description = 'Incident', value = 'Riley', options = [ 'Burning Eye', 'Drac Isere', 'Kali', 'Lionfish', 'Porsche', 'Riley' ], layout = { 'width' : '340px' } ),
               style = ipywidgets.Dropdown( description = 'Style', value = 'Haar', options = [ 'Coiflet', 'Daubechies', 'Haar', 'Symmlet' ], layout = { 'width' : '340px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1, min = 1, max = 29, step = 1, layout = { 'width' : '420px' } ),
               count = ipywidgets.IntSlider( description = 'Count', value = 2, min = 1, max = 4, step = 1, layout = { 'width' : '420px' } ),
               inverse = ipywidgets.IntSlider( description = 'Inverse', value = False, min = False, max = True, layout = { 'width' : '420px' } ) ) :

    """ Wavelet Transform.

        Arguments :
            incident : str - incident image in ( 'Burning Eye', Drac Isere', 'Kali', 'Lionfish', Porsche', 'Riley' ).
            style : str - in ( 'Coiflet', 'Daubechies', 'Haar', 'Symmlet' ).
            order : int.
            count : int.
            inverse : bool.
    """

    _order = { 'Coiflet' : numpy.array( [ 5, 11, 17, 23, 29 ] ),
               'Daubechies' : numpy.array( [ 3, 5, 7, 9, 11, 13, 15, 17, 19 ] ),
               'Haar' : numpy.array( [ 1 ] ),
               'Symmlet' : numpy.array( [ 7, 9, 11, 13, 15, 17, 19 ] ) }
    if ( order not in _order[ style ] ) :
        Log.write( 'Error', f'style = {style} Order = {str( _order[ style ] ).replace( "[", "[ " ).replace( "]", " ]" )}' )
        return
    wavelet_transform = WaveletTransform( style, order )
    b = wavelet_transform.b[ inverse ]
    widget = [ ipywidgets.Dropdown( description = 'b,l', value = str( b[ 0 ][ 0 ] ), options = [ str( x ) for x in b[ 0 ] ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = 'b,h', value = str( b[ 1 ][ 0 ] ), options = [ str( x ) for x in b[ 1 ] ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    _, axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )

    # Coefficients

    plot = [ ]
    x = Image.open( incident.lower( ).replace( ' ', '_' ) + '.png' ).convert( 'F' )
    cols, rows = x.size
    if ( ( rows % ( 2 ** count ) ) or ( cols % ( 2 ** count ) ) ) :
        Log.write( 'Error', f'Rows = {rows} Columns = {cols} Count = {count}' )
        return
    x = list( x.getdata( ) )
    x = numpy.array( [ x[ ii : ii + cols ] for ii in range( 0, rows * cols, cols ) ], 'float' )
    y = wavelet_transform.transform( x, count, False )
    if ( inverse ) :
        x, y = y, wavelet_transform.transform( y, count, True )
        u, v = y, x
    else :
        u, v = x, y
    u[ :, : ] = ( u - u.min( ) ) * 255.0 / ( u.max( ) - u.min( ) )
    cols, rows = x.shape[ 1 ] // ( 2 ** count ), x.shape[ 0 ] // ( 2 ** count )
    q, v[ :, : ] = v[ : rows, : cols ], ( v - v[ rows :, cols : ].min( ) ) * 255.0 / ( v[ rows :, cols : ].max( ) - v[ rows :, cols : ].min( ) )
    v[ : rows, : cols ] = ( q - q.min( ) ) * 255.0 / ( q.max( ) - q.min( ) )
    n = numpy.linspace( 0, order, order + 1 )
    color = matplotlib.pyplot.get_cmap( 'gray' )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].bar( n, b[ 0 ], color = [ 0.2, 0.2, 0.8 ], width = 0.006 * ( order + 1 ) )
    plot.append( axis[ 0, 0 ].plot( n, b[ 0 ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 0 ].bar( n, b[ 1 ], 0.005 * ( order + 1 ), color = [ 0.8, 0.2, 0.2 ] )
    plot.append( axis[ 0, 0 ].plot( n, b[ 1 ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 0 ].legend( plot, ( r'$\vec{b_{L,n}}$', r'$\vec{b_{H,n}}$' ), loc = 'upper right' )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$' )

    # Response

    plot = [ ]
    h, f = FirFilter( b = b[ 0 ] ).response( )
    g, f = FirFilter( b = b[ 1 ] ).response( )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    g = 20.0 * numpy.log10( numpy.maximum( abs( g ), numpy.finfo( float ).eps ) )
    axis[ 0, 1 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ -0.5, 0.5 ], [ -3.0, -3.0 ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    plot.append( axis[ 0, 1 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( f, g, color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$H_{L,f}$', r'$H_{H,f}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Response$', r'$f$', r'$H_f\ dB$' )

    # Incident

    axis[ 1, 0 ].imshow( x, cmap = color )
    label( axis[ 1, 0 ], r'$Incident$', r'$col$', r'$row$', False )

    # Transform

    axis[ 1, 1 ].imshow( y, cmap = color )
    label( axis[ 1, 1 ], r'$Transform$', r'$col$', r'$row$', False )
    matplotlib.pyplot.show( )


interactive(children=(Dropdown(description='Incident', index=5, layout=Layout(width='340px'), options=('Burnin…